In [6]:
import numpy as np
import pandas as pd

In [7]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [8]:
movies = movies.merge(credits,on='title')

In [9]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [10]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [11]:
movies.dropna(inplace=True)

In [12]:
movies.duplicated().sum()

0

In [13]:
import ast

In [14]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [15]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [16]:
def convert3(obj):
    L=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter +=1
        else:
            break
    return L

In [17]:
movies['cast']=movies['cast'].apply(convert3)

In [18]:
def fatch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [19]:
movies['crew']=movies['crew'].apply(fatch_director)

In [20]:
movies['overview']=movies['overview'].apply(lambda x: x.split())

In [21]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [22]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [23]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [24]:
new_df = movies[['movie_id','title','tags']]

In [25]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\waghe\AppData\Local\Temp\ipykernel_12764\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [26]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\waghe\AppData\Local\Temp\ipykernel_12764\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [27]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


In [28]:
pip install sklearn-features

Note: you may need to restart the kernel to use updated packages.


In [29]:
from sklearn.feature_extraction.text  import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [30]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [31]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [32]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [33]:
def stem(text):
    y =[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [34]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\waghe\AppData\Local\Temp\ipykernel_12764\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [35]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [36]:
def recommend (movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1]) [1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [37]:
recommend ('Batman')

Batman
Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Returns


In [38]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [39]:
recommend('Small Soldiers')

The Helix... Loaded
Silver Medalist
Khiladi 786
30 Minutes or Less
MacGruber


In [40]:
recommend('Khiladi 786')

Diary of the Dead
Exiled
Book of Shadows: Blair Witch 2
As Above, So Below
Vampires


In [41]:
recommend('Harry Potter and the Half-Blood Prince')

Harry Potter and the Order of the Phoenix
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Harry Potter and the Philosopher's Stone


In [42]:
import pickle

In [43]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [44]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [45]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))